In [2]:
using Revise
using Bilevel

using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Plots
using LinearAlgebra
using StaticArrays

gr(size=(800,500),html_output_format=:png,
    seriescolor=:magma,background_color_legend=:white,
    background=RGB(((240,240,240) ./ 255.)...));

# Contact

## Single point with contact

In [ ]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), SVector([0.,0.,0.]...))
normal = FreeVector3D(default_frame(floor), SVector([0.,0.,1.]...))
floor_obs = Obstacle(floor, point, normal, :xyz, 1.)

obstacles = [floor_obs]
env = Environment(mechanism, urdf, obstacles)

In [ ]:
q1 = [1., 0., 0., 0., 0., 0., 0.]
qN = [1., 0., 0., 0., .1, 0., 0.]
N = 5
Δt = .005

function add_prob_constraints!(sim_data)
    vs = sim_data.vs

    add_eq!(sim_data, :cq1, length(q1), x -> vs(x, :q1) - q1)
    add_eq!(sim_data, :cqN, length(qN), x -> vs(x, Symbol("q", N)) - qN)

    add_box_con!(sim_data, :u, :u, zeros(num_velocities(mechanism)), zeros(num_velocities(mechanism)), 1:N-1)
    add_box_con!(sim_data, :hbox, :h, .5 * [Δt], 2. * [Δt], 1:N-1)
end

In [ ]:
sim_data = get_trajopt_data_indirect(mechanism,env,Δt,N,relax_comp=false)
add_prob_constraints!(sim_data)
traj_indirect = Bilevel.trajopt(sim_data, quaternion_state=true, verbose=1)

In [ ]:
sim_data = get_trajopt_data_semidirect(mechanism,env,Δt,N,relax_comp=false)
add_prob_constraints!(sim_data)
traj_semidirect = Bilevel.trajopt(sim_data, quaternion_state=true, verbose=1)

In [ ]:
sim_data = get_trajopt_data_direct(mechanism,env,Δt,N,relax_comp=false)
add_prob_constraints!(sim_data)
traj_direct = Bilevel.trajopt(sim_data, quaternion_state=true, verbose=1)

In [ ]:
j = 7
plot(traj_indirect[7],traj_indirect[8][j,:],label="Indirect",seriescolor=RGB(((227,74,51) ./ 255.)...),linewidth=4,marker=6)
plot!(traj_semidirect[7],traj_semidirect[8][j,:],label="Semidirect",seriescolor=RGB(((254,232,200) ./ 255.)...),linewidth=0,marker=4)
plot!(traj_direct[7],traj_direct[8][j,:],label="Direct",seriescolor=RGB(((224,132,200) ./ 255.)...),linewidth=0,marker=4)

## Little dog

In [3]:
urdf = joinpath("..", "urdf", "little_dog", "little_dog.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), SVector([0.,0.,0.]...))
normal = FreeVector3D(default_frame(floor), SVector([0.,0.,1.]...))
floor_obs = Obstacle(floor, point, normal, :xyz, 1.)

obstacles = [floor_obs]
env = Environment(mechanism, urdf, obstacles);

In [15]:
q_nom = [1., 0., 0., 0., 0., 0., .175, 0., 0., 0., 0., .036, 0., 0., 0., .00758, 0., .002445, 0.]  
q_min = [-1., -1., -1., -1., -.1, -.01, .05, -.6, -.6, -.6, -.6, -3.5, -3.5, -3.5, -3.5, -3.1, -3.1, -3.1, -3.1]
q_max = [1., 1., 1., 1., 5., .01, .2, .6, .6, .6, .6, 2.4, 2.4, 2.4, 2.4, 1., 1., 1., 1.]

N = 10
Δt = .05

function add_prob_constraints!(sim_data)
    vs = sim_data.vs

    add_eq!(sim_data, :cq1, num_positions(mechanism), x -> vs(x, :q1) - q_nom)
    add_eq!(sim_data, :cv1, num_velocities(mechanism), x -> vs(x, :v1))
end

function add_prob_obj!(sim_data)
    add_obj!(sim_data, :xf, x -> -sim_data.vs(x, Symbol("q", N))[5])
end

function get_prob_limits(sim_data)
    x_min = -1e19*ones(sim_data.vs.num_vars)
    x_max = 1e19*ones(sim_data.vs.num_vars)
    
    # joint limits
    add_box_con_snopt!(x_min, x_max, sim_data, :q, q_min, q_max, 1:N)
    add_box_con_snopt!(x_min, x_max, sim_data, :h, [.5 * Δt], [2. * Δt], 1:N-1)
    
    # torques limits
    u_limit = .5
    add_box_con_snopt!(x_min, x_max, sim_data, :u, -u_limit * ones(num_velocities(mechanism)), u_limit * ones(num_velocities(mechanism)), 1:N-1)

    x_min, x_max
end

function get_prob_init(sim_data)
    x0 = zeros(sim_data.vs.num_vars)
    for n = 1:N
        x0[sim_data.vs(Symbol("q", n))] .= q_nom
    end
end

function setup_prob!(sim_data)
    add_prob_constraints!(sim_data)
    add_prob_obj!(sim_data)
    x_min, x_max = get_prob_limits(sim_data)
    x0 = get_prob_init(sim_data)
    
    x0, x_min, x_max
end

setup_prob! (generic function with 1 method)

In [18]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf));
open(mvis)

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8701
└ @ MeshCat /Users/blandry/.julia/packages/MeshCat/WlA0B/src/servers.jl:24


Process(`open http://127.0.0.1:8701`, ProcessExited(0))

In [16]:
sim_data = get_trajopt_data_indirect(mechanism,env,Δt,N,relax_comp=true)
x0, x_min, x_max = setup_prob!(sim_data)
traj_indirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=1e-3, major_feas=1e-3, minor_feas=1e-3);

Resource limit error: iteration limit reached


In [19]:
setanimation!(mvis, traj_indirect[7], traj_indirect[1])

In [ ]:
sim_data = get_trajopt_data_semidirect(mechanism,env,Δt,N,relax_comp=true)
x0, x_min, x_max = setup_prob!(sim_data)
traj_semidirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=1e-3, major_feas=1e-3, minor_feas=1e-3);

In [ ]:
setanimation!(mvis, traj_semidirect[7], traj_semidirect[1])